# Clase Práctica 2: Validación de modelos 📈

----------------------------------

Los modelos de regresión corresponden a un subcampo del aprendizaje supervisado que busca modelar la relación entre un cierto número de características y una variable objetivo continua. La calidad del modelo quedará determinada por la distancia entre el valor real y el predicho.

El clásico ejemplo es el modelo de regresión lineal, el cual es un algoritmo estadístico que busca establecer la recta que muestra la tendencia en un conjunto de datos. 

Estos modelos pueden ser aplicados a cualquier problema que busque predecir un número real. Por ejemplo, predecir el precio de una casa, el costo de un tratamiento, el número de días en que un paciente será dado de alta, entre muchas más tareas.






## Objetivos de la clase 📚

Los objetivos principales de esta clase son los siguientes:



1.   Explorar varios modelos de regresión.
2.   Explorar varias métricas.
3.   Determinar los modelos que mejor se adaptan al problema.


Específicamente, utilizaremos un conjunto de datos que contiene valores de seguros de salud según características del paciente.

**Lectura del dataset**

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/fvillena/biocompu/2022/data/insurance.csv')

Entonces, lo que haremos será analizar este dataset que está dedicado al costo de tratamiento de diversos pacientes. Si bien el costo de tratamiento depende de muchos factores médicos, existen otras variables que sería interesante explorar como la edad, sexo, ubicación, entre otras.

Seleccionamos 3 filas al azar para ver de qué se tratan los datos

In [ ]:
df.sample(3)

Vemos los nombres de las columnas y los tipos de datos

In [ ]:
df.info()

Veamos si existen valores nulos


In [ ]:
df.isnull().sum()

Lo siguiente que haremos será analizar cuáles son las variables que están más correlacionadas con el costo del tratamiento. Pero antes, hablemos un poco sobre las variables categóricas. Estas variables contienen un número finito de posibilidades, y pueden ser tanto strings como números. Como vimos en la clase anterior, si es que tenemos variables de tipo strings, debemos realizar una transformación.

In [ ]:
df.sample(1)

Del resultado anterior vemos que las variables categoricas serían sexo, fumador y región. Para poder transformarlas podemos utilizar la clase LabelEncoder.

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Transformamos la variable sexo
df.age

In [ ]:
le = LabelEncoder()

print(df.sex.drop_duplicates())
le.fit(df.sex.drop_duplicates()) 
df.sex[0]

In [ ]:
df.sex = le.transform(df.sex)

In [ ]:
le.fit(df.smoker.drop_duplicates()) 
df.smoker = le.transform(df.smoker)
df.smoker

In [ ]:
le.fit(df.region.drop_duplicates()) 
df.region = le.transform(df.region)
df.region

Entonces, básicamente lo que hace esta clase es transformar cada valor de la columna en un valor numérico mediante un mapeo. En el caso de sexo, sabemos que los valores posible son *Male* y *Female*, por lo tanto, le asignamos un 0 al primer valor y un 1 al segundo. Aquí también pueden utilizar la función get_dummies vista en la clase anterior, pero siempre está el riesgo de aumentar considerablemente la cantidad de features. Ahora veamos como está la correlación de los datos.

In [ ]:
df.corr()['charges'].sort_values()

A priori podemos ver que la variable más relacionada con el costo del tratamiento es si el paciente es fumador o no, mientras que la que menos se relaciona es la región. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = df.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(240,10,as_cmap=True),
            square=True, ax=ax)

Si seguimos analizando la variable smoker, en el siguiente gráfico podemos ver la distribución de costos de las personas fumadoras versus los no fumadores.

In [ ]:
f= plt.figure(figsize=(12,5))

ax=f.add_subplot(121)
sns.distplot(df[(df.smoker == 1)]["charges"],color='c',ax=ax)
ax.set_title('Distribution of charges for smokers')

ax=f.add_subplot(122)
sns.distplot(df[(df.smoker == 0)]['charges'],color='b',ax=ax)
ax.set_title('Distribution of charges for non-smokers')

Si bien el costo del tratamiento de los fumadores es mucho más elevados, es importante ver la distribución de frecuencias de estos datos.

In [ ]:
sns.catplot(x="smoker", kind="count",hue = 'sex', palette="pink", data=df)

Veamos un box plot a ver si encontramos algo más interesante acerca de los fumadores versus no fumadores.

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Box plot for charges of women")
sns.boxplot(y="smoker", x="charges", data =  df[(df.sex == 0)] , orient="h", palette = 'magma')

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Box plot for charges of men")
sns.boxplot(y="smoker", x="charges", data =  df[(df.sex == 1)] , orient="h", palette = 'rainbow')

Y si queremos ver el costo de tratamiento en personas de 18 años según si fuman o no?

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Box plot for charges 18 years old smokers")
sns.boxplot(y="smoker", x="charges", data = df[(df.age == 18)] , orient="h", palette = 'pink')

Interesante, de todas maneras el objetivo de este práctico no es la visualización y análisis así que pasemos a la parte de los modelos.

# **Selección de modelos**

Siguiendo la receta de la primera clase, lo primero que hacemos es dividir nuestro conjunto de datos original en las particiones X e y, donde X corresponde a las características que serán pasadas al modelo, mientras que y es la variable objetivo.

In [ ]:
X = df.drop(['charges'], axis = 1)
y = df.charges

Luego, para poder medir bien el nivel de generalización del modelo, dividimos el conjunto de datos en entrenamiento y testing.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 123)

**Regresión Lineal**

Esta es una de las técnicas de regresión más utilizadas en aprendizaje de máquinas. Como se vió en clases, la idea general de este algoritmo en encontrar al recta que mejor se ajusta a un conjunto de datos continuos.

En el caso más simple, cuando queremos describir una variable dependiente `y` según una variable independiente `x`, el problema se reduce a la siguiente ecuación:

$$y = wx + b$$

En el caso general tendremos un conjunto de n variables independientes (features) y 1 variable dependiente, de manera que la ecuación quedará definida como:

$$y = w_{0}x_{0} + w_{1}x_{1} + ... +. w_{n}x_{n} + b$$

El proceso de aprendizaje consiste en encontrar qué conjunto de parámetros $w_{0}, .... , w_{n}$ minimiza el error cuadrático medio entre los valores predichos y los reales.


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression().fit(X_train, y_train)

Luego de entrenar nuestro modelo podemos ver cuáles de los coeficientes son los que más pesan.

In [ ]:
lr_vil = pd.DataFrame(lr.coef_,columns=["value"]) # Guardamos en un dataframe los coeficientes
lr_vil.index = df.columns[:-1]
lr_vil

In [ ]:
y_pred = lr.predict(X_test)

Lo siguiente que debemos hacer es definir cuáles son las métricas que utilizaremos para medir nuestro modelo. En este caso usaremos el error absoluto medio, la raíz del error cuadrático medio, y el coeficiente de determinación, que se definen con las siguientes fórmulas.

$MAE = \frac{1}{m}\sum_{i=1}^{m}|y_{true}-y_{pred}|$

$RSME = \frac{1}{m}\sum_{i=1}^{m}(y_{true}-y_{pred})^2$

$R^2 = 1 - \frac{\frac{1}{m}\sum_{i=1}^{m}(y_{true}-y_{pred})^2}{\frac{1}{m}\sum_{i=1}^{m}(y_{true}-y_{mean})^2}$

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
 
def compute_metrics(y_true, y_pred):
    """
    Esta función recibe un arreglo de valores reales y predichos para 
    retornar un diccionario con una serie de métricas de regresión
    """
    return {
        'mae': mean_absolute_error(y_true, y_pred),
        'rmse': mean_squared_error(y_true, y_pred) ** 0.5,
        'r2': r2_score(y_true, y_pred)
    }

In [ ]:
lr_regression_report = compute_metrics(y_test, y_pred)

In [ ]:
lr_regression_report

**Random Forest Regressor**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
rf_vil = pd.DataFrame(list(zip(df.columns[:-1],rf.feature_importances_)),
             columns=["feature","importance"]
            ).set_index("feature")
rf_vil.sort_values("importance",ascending=False)

In [ ]:
rf_regression_report = compute_metrics(y_test, y_pred)
rf_regression_report

**Support Vector Regressor**

In [ ]:
from sklearn.svm import SVR

svr = SVR(kernel="linear", C=1.0)
svr.fit(X_train, y_train)

In [ ]:
y_pred = svr.predict(X_test)

In [ ]:
svr_regression_report = compute_metrics(y_test, y_pred)
svr_regression_report

In [ ]:
svr_vil = pd.DataFrame(list(zip(df.columns[:-1],svr.coef_[0])),
             columns=["feature","importance"]
            ).set_index("feature")
svr_vil.sort_values("importance",ascending=False)

**K-Nearest Neighbor Regressor**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor() # Instanciamos una support vector machine con un kernel lineal
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
knn_regression_report = compute_metrics(y_test, y_pred)
knn_regression_report

In [ ]:
performances = pd.DataFrame( # Consolidamos todas las métricas en un DatFrame
    data = [
        lr_regression_report,
        knn_regression_report,
        rf_regression_report,
        svr_regression_report
    ],
    index = [
        "Linear Regression",
        "k-Nearest Neighbors",
        "Random Forest",
        "SVR"
    ]
).sort_values( # Ordenamos los valores
    by="rmse"
)
performances

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
cb = CatBoostRegressor(learning_rate =0.01, max_depth =5, verbose = 0)

cb.fit(X_train, y_train)

In [ ]:
y_pred = cb.predict(X_test)

In [ ]:
compute_metrics(y_test, y_pred)